In [66]:
# cargo librerias
import pandas as pd
import re as re
import numpy as np

### Correspondencia de productos

World Integrated Trade Solution (WITS) ofrece información sobre diversas nomenclaturas de productos, lo que contribuye con el mapeo entre varias nomenclaturas de productos. En este caso se descargó la tabla de correlación entre el SA 1996 o H1 y la ISIC Rev 3, ya que los datos de exportaciones se encuentran en dicha nomenclatura del SA y los de clasificación de bienes pertenecientes a la Bioeconomía en la mencionada clasificación ISIC. https://wits.worldbank.org/product_concordance.html

#### Lista de productos pertenecientes a la bioeconomía en CIIU Rev3

In [25]:
# Bienes pertenecientes a la bioeconomia argentina
CIIU_bioecon = pd.read_excel('../data/raw/Nomenclaturas/clasif_CIIUrev3_bioecon.xlsx', index_col=None)  
CIIU_bioecon = CIIU_bioecon.loc[ 3:79 , : ]

In [26]:
name_columns = CIIU_bioecon.loc[ 3 , : ].values
name_columns[0]

'CIIU Rev3'

In [27]:
CIIU_bioecon = CIIU_bioecon.loc[ 4:79 , : ].rename(columns = {'Actividades consideradas para estimar la bioeconomía': name_columns[0], 'Unnamed: 1': name_columns[1], 'Unnamed: 2': name_columns[2]})

In [28]:
CIIU_bioecon

,CIIU Rev3,Concepto,Participación %
4,Letra A,"Agricultura, ganadería, caza y silvicultura",100
5,Letra B,Pesca,100
6,Letra D,Industria Manufacturera Bio,NaN
7,15111,"Matanza de ganado, producción, procesamiento y...",100
8,15112,"Producción, procesamiento y conservación de ca...",100
...,...,...,...
75,24241,Fabricación de jabones y preparados de limpieza,50
76,24249,"Fabricación de cosméticos, perfumes y otros pr...",25
77,24290,Fabricación de productos químicos n.c.p. (BIOD...,100
78,36101,"Fabricación de muebles y partes de muebles, pr...",100


De las letras A y B se consideran todos los productos como productos de la Bioeconomía, mientras que de la letra D sólo los que se mencionan ahí en los porcentajes que allí indican. En este caso, si figura en el listado se considerará parte de la bioeconomía aunque no corresponda en su totalidad dado que no hay forma de separarlos. 

Voy a tener que extraer del listado completo de nomenclatura CIIU Rev3 los códigos numéricos de productos correspondientes a las letras A y B que no figuran en el listado previo. 

In [29]:
# dejo afuera las filas correspondientes a letras (A, B y D) y me quedo sólo con los códigos numéricos
CIIU_bioecon = CIIU_bioecon.loc[ 7:79 , : ]

In [30]:
CIIU_bioecon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 7 to 79
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CIIU Rev3        73 non-null     object
 1   Concepto         73 non-null     object
 2   Participación %  73 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


#### Lista total de productos en nomenclatura CIIU Rev3

In [11]:
# descargo datos completos de nomenclatura CIIU_Rev3 para completar las referencias a letras
CIIU_Rev3 = pd.read_excel('../data/raw/Nomenclaturas/CIIU_Rev3_es.xlsx')  
CIIU_Rev3

,Code,Description,Categoria
0,A,"Agricultura, ganadería, caza y silvicultura",NaN
1,1,"Agricultura, ganadería, caza y actividades de ...",A
2,11,Cultivos en general; cultivo de productos de m...,A
3,111,Cultivo de cereales y otros cultivos n.c.p.,A
4,112,"Cultivo de hortalizas y legumbres, especialida...",A
...,...,...,...
523,9500,Hogares privados con servicio doméstico,P
524,Q,Organizaciones y órganos extraterritoriales,NaN
525,99,Organizaciones y órganos extraterritoriales,Q
526,990,Organizaciones y órganos extraterritoriales,Q


In [47]:
# se corrobora que la clasificación CIIU Rev3 tiene como máximo hasta 4 dígitos
CIIU_Rev3.Code.astype(str).str.len().max()

4

Por lo que se pudo observar en la clasificación CIIU Rev3 los códigos tienen solo 4 dígitos, por lo que se descubrió que la clasificación que en el trabajo de bioeconomía figura como CIIU Rev3 en realidad corresponde a la [Clasificación Nacional de Actividades Económicas (ClaNAE-97)](https://www.argentina.gob.ar/sites/default/files/clasificacion_nacional_de_actividades_economicas-clanae-97.pdf) de Argentina. Según se pudo verificar en el link de dicha clasificación que compara con la CIIU Rev3, la diferencia está en el 5to número, mientras los primeros 4 coinciden con la clasificación CIIU Rev3. Por lo que en este caso se procederá a seleccionar los primeros 4 dígitos de dicha columna para llegar a obtener la clasificación CIIU Rev3 efectiva. 

In [51]:
# selecciono los datos correspondientes a letras A y B de esta base
CIIU_Rev3[CIIU_Rev3.Categoria.isin(['A','B'])]

,Code,Description,Categoria
1,1,"Agricultura, ganadería, caza y actividades de ...",A
2,11,Cultivos en general; cultivo de productos de m...,A
3,111,Cultivo de cereales y otros cultivos n.c.p.,A
4,112,"Cultivo de hortalizas y legumbres, especialida...",A
5,113,"Cultivo de frutas, nueces, plantas cuyas hojas...",A
6,12,Cría de animales,A
7,121,"Cría de ganado vacuno y de ovejas, cabras, cab...",A
8,122,Cría de otros animales; elaboración de product...,A
9,13,Cultivo de productos agrícolas en combinación ...,A
10,130,Cultivo de productos agrícolas en combinación ...,A


In [61]:
# selecciono los códigos correspondientes a letras A y B
codigos_AB = CIIU_Rev3[CIIU_Rev3.Categoria.isin(['A','B'])].Code
codigos_AB = codigos_AB.astype(str).unique()
codigos_AB

array(['1', '11', '111', '112', '113', '12', '121', '122', '13', '130',
       '14', '140', '15', '150', '2', '20', '200', '5', '50', '500'],
      dtype=object)

Para complementar los códigos de las letras A y B, selecciono las posiciones correspondientes a la letra D de la tabla CIIU_bioecon 

In [34]:
CIIU_bioecon['CIIU Rev3'] = CIIU_bioecon['CIIU Rev3'].astype(str)

<ipython-input-34-15aef9966eba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CIIU_bioecon['CIIU Rev3'] = CIIU_bioecon['CIIU Rev3'].astype(str)


In [38]:
CIIU_bioecon['CIIU_4dig'] = CIIU_bioecon['CIIU Rev3'].str[:4]

<ipython-input-38-030a28f411fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CIIU_bioecon['CIIU_4dig'] = CIIU_bioecon['CIIU Rev3'].str[:4]


In [39]:
CIIU_bioecon

,CIIU Rev3,Concepto,Participación %,CIIU_4dig
7,15111,"Matanza de ganado, producción, procesamiento y...",100,1511
8,15112,"Producción, procesamiento y conservación de ca...",100,1511
9,15113,Elaboración de fiambres y embutidos,100,1511
10,15120,Elaboración y conservación de pescado y produc...,100,1512
11,15130,"Elaboración y conservación de frutas, legumbre...",100,1513
...,...,...,...,...
75,24241,Fabricación de jabones y preparados de limpieza,50,2424
76,24249,"Fabricación de cosméticos, perfumes y otros pr...",25,2424
77,24290,Fabricación de productos químicos n.c.p. (BIOD...,100,2429
78,36101,"Fabricación de muebles y partes de muebles, pr...",100,3610


In [64]:
# códigos de CIIU que corresponden a la letra D
codigos_D = CIIU_bioecon.CIIU_4dig.unique() 
codigos_D

array(['1511', '1512', '1513', '1514', '1520', '1531', '1532', '1533',
       '1541', '1542', '1543', '1544', '1549', '1551', '1552', '1553',
       '1554', '1600', '1711', '1712', '1721', '1722', '1723', '1729',
       '1730', '1810', '1820', '1911', '1912', '1920', '2010', '2021',
       '2022', '2023', '2029', '2101', '2102', '2109', '2320', '2411',
       '2421', '2423', '2424', '2429', '3610', '3720'], dtype=object)

In [74]:
# uno el listado de códigos correspondientes a la bioeconomia en CIIU Rev3
list_CIIU_bioecon = list(codigos_AB) + list(codigos_D) 
list_CIIU_bioecon = [int(i) for i in list_CIIU_bioecon] # los paso a número
list_CIIU_bioecon

[1,
 11,
 111,
 112,
 113,
 12,
 121,
 122,
 13,
 130,
 14,
 140,
 15,
 150,
 2,
 20,
 200,
 5,
 50,
 500,
 1511,
 1512,
 1513,
 1514,
 1520,
 1531,
 1532,
 1533,
 1541,
 1542,
 1543,
 1544,
 1549,
 1551,
 1552,
 1553,
 1554,
 1600,
 1711,
 1712,
 1721,
 1722,
 1723,
 1729,
 1730,
 1810,
 1820,
 1911,
 1912,
 1920,
 2010,
 2021,
 2022,
 2023,
 2029,
 2101,
 2102,
 2109,
 2320,
 2411,
 2421,
 2423,
 2424,
 2429,
 3610,
 3720]

#### Tabla de correspondencias CIIU Rev3 y HS 1996

In [12]:
correl_nomen = pd.read_csv('../data/raw/Nomenclaturas/JobID-19_Concordance_H1_to_I3.csv') 
correl_nomen

,HS 1996 Product Code,HS 1996 Product Description,ISIC Revision 3 Product Code,ISIC Revision 3 Product Description
0,10111,Horses :-- Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses..."
1,10119,Horses :-- Other,121,"Farming of cattle, sheep, goats, horses, asses..."
2,10120,"Asses, mules and hinnies",121,"Farming of cattle, sheep, goats, horses, asses..."
3,10210,Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses..."
4,10290,Other,121,"Farming of cattle, sheep, goats, horses, asses..."
...,...,...,...,...
5108,970200,"Original engravings, prints and lithographs.",9214,"Dramatic arts, music and other arts activities"
5109,970300,"Original sculptures and statuary, in any mater...",9214,"Dramatic arts, music and other arts activities"
5110,970400,"Postage or revenue stamps, stamp-postmarks, fi...",9214,"Dramatic arts, music and other arts activities"
5111,970500,Collections and collectors' pieces of zoologic...,9214,"Dramatic arts, music and other arts activities"


In [13]:
# verifico con un ejemplo para el codigo rev3 1511 que inlcuye 77 productos a 6 dígitos
correl_nomen[correl_nomen['ISIC Revision 3 Product Code']==1511]

,HS 1996 Product Code,HS 1996 Product Description,ISIC Revision 3 Product Code,ISIC Revision 3 Product Description
17,20110,Carcasses and half-carcasses,1511,"Production, processing and preserving of meat ..."
18,20120,Other cuts with bone in,1511,"Production, processing and preserving of meat ..."
19,20130,Boneless,1511,"Production, processing and preserving of meat ..."
20,20210,Carcasses and half-carcasses,1511,"Production, processing and preserving of meat ..."
21,20220,Other cuts with bone in,1511,"Production, processing and preserving of meat ..."
...,...,...,...,...
1841,410210,With wool on,1511,"Production, processing and preserving of meat ..."
1842,410221,Without wool on :-- Pickled,1511,"Production, processing and preserving of meat ..."
1843,410229,Without wool on :-- Other,1511,"Production, processing and preserving of meat ..."
1844,410310,Of goats or kids,1511,"Production, processing and preserving of meat ..."


In [78]:
correl_nomen = correl_nomen.rename(columns={'HS 1996 Product Code': 'HS_code', 'HS 1996 Product Description': 'HS_description', 'ISIC Revision 3 Product Code':'CIIU_Rev3_code', 'ISIC Revision 3 Product Description':'CIIU_Rev3_desc'})
correl_nomen

,HS_code,HS_description,CIIU_Rev3_code,CIIU_Rev3_desc
0,10111,Horses :-- Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses..."
1,10119,Horses :-- Other,121,"Farming of cattle, sheep, goats, horses, asses..."
2,10120,"Asses, mules and hinnies",121,"Farming of cattle, sheep, goats, horses, asses..."
3,10210,Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses..."
4,10290,Other,121,"Farming of cattle, sheep, goats, horses, asses..."
...,...,...,...,...
5108,970200,"Original engravings, prints and lithographs.",9214,"Dramatic arts, music and other arts activities"
5109,970300,"Original sculptures and statuary, in any mater...",9214,"Dramatic arts, music and other arts activities"
5110,970400,"Postage or revenue stamps, stamp-postmarks, fi...",9214,"Dramatic arts, music and other arts activities"
5111,970500,Collections and collectors' pieces of zoologic...,9214,"Dramatic arts, music and other arts activities"


Los datos a nivel de producto se encuentran a 6 dígitos del SA, por lo que voy a tener que convertir a nivel de partida arancelaria (4 dígitos) para que coincida con los datos de comercio con que se está trabajando. 

In [87]:
# genero una nueva columna que considere nivel de partida arancelaria (4 dígitos) en vez de 6 digtios del SA (HS)
correl_nomen['HS_4dig'] = np.where(correl_nomen.HS_code.astype(str).str.len() == 5, correl_nomen.HS_code.astype(str).str[:3], correl_nomen.HS_code.astype(str).str[:4]) 
correl_nomen

,HS_code,HS_description,CIIU_Rev3_code,CIIU_Rev3_desc,HS_4dig
0,10111,Horses :-- Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses...",101
1,10119,Horses :-- Other,121,"Farming of cattle, sheep, goats, horses, asses...",101
2,10120,"Asses, mules and hinnies",121,"Farming of cattle, sheep, goats, horses, asses...",101
3,10210,Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses...",102
4,10290,Other,121,"Farming of cattle, sheep, goats, horses, asses...",102
...,...,...,...,...,...
5108,970200,"Original engravings, prints and lithographs.",9214,"Dramatic arts, music and other arts activities",9702
5109,970300,"Original sculptures and statuary, in any mater...",9214,"Dramatic arts, music and other arts activities",9703
5110,970400,"Postage or revenue stamps, stamp-postmarks, fi...",9214,"Dramatic arts, music and other arts activities",9704
5111,970500,Collections and collectors' pieces of zoologic...,9214,"Dramatic arts, music and other arts activities",9705


#### Selecciono productos pertenecientes a la bioeconomía de la tabla de correspondencias

In [89]:
# selecciono sólo los bienes que pertenecen al listado de la bioeconomía
df_bioecon = correl_nomen[correl_nomen.CIIU_Rev3_code.isin(list_CIIU_bioecon)]
df_bioecon

,HS_code,HS_description,CIIU_Rev3_code,CIIU_Rev3_desc,HS_4dig
0,10111,Horses :-- Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses...",101
1,10119,Horses :-- Other,121,"Farming of cattle, sheep, goats, horses, asses...",101
2,10120,"Asses, mules and hinnies",121,"Farming of cattle, sheep, goats, horses, asses...",101
3,10210,Pure-bred breeding animals,121,"Farming of cattle, sheep, goats, horses, asses...",102
4,10290,Other,121,"Farming of cattle, sheep, goats, horses, asses...",102
...,...,...,...,...,...
4999,940421,"Mattresses :-- Of cellular rubber or plastics,...",3610,Manufacture of furniture,9404
5000,940429,Mattresses :-- Of other materials,3610,Manufacture of furniture,9404
5001,940430,Sleeping bags,1721,"Manufacture of made-up textile articles, excep...",9404
5002,940490,Other,1721,"Manufacture of made-up textile articles, excep...",9404


Una vez filtrados por código CIIU, selecciono los códigos únicos a 4 dígitos del SA (clasificación de datos de comercio) para poder clasificar los datos de comercio en bienes de la bioeconomía o no. 

In [92]:
# armo lista de códigos HS
list_bioecon_HS4dig = df_bioecon.HS_4dig.unique()
list_bioecon_HS4dig

array(['101', '102', '103', '104', '105', '106', '201', '202', '203',
       '204', '205', '206', '207', '208', '209', '210', '301', '302',
       '303', '304', '305', '306', '307', '401', '402', '403', '404',
       '405', '406', '407', '408', '409', '410', '503', '508', '509',
       '511', '601', '602', '603', '604', '701', '702', '703', '704',
       '705', '706', '707', '708', '709', '710', '711', '712', '713',
       '714', '801', '802', '803', '804', '805', '806', '807', '808',
       '809', '810', '811', '812', '813', '814', '901', '902', '903',
       '904', '905', '906', '907', '908', '909', '910', '1001', '1002',
       '1003', '1004', '1005', '1006', '1007', '1008', '1101', '1102',
       '1103', '1104', '1105', '1106', '1107', '1108', '1109', '1201',
       '1202', '1203', '1204', '1205', '1206', '1207', '1208', '1209',
       '1210', '1211', '1212', '1213', '1214', '1301', '1302', '1401',
       '1402', '1403', '1404', '1501', '1502', '1503', '1504', '1505',
       '1506'

In [96]:
print('Número de bienes en código HS_4dig que corresponden a la bioeconomía: ', len(list_bioecon_HS4dig.astype(int))) # 580 bienes corresponden a la bioeconomía 
list_bioecon_HS4dig = list(list_bioecon_HS4dig.astype(int))
list_bioecon_HS4dig

Número de bienes en código HS_4dig que corresponden a la bioeconomía:  580


[101,
 102,
 103,
 104,
 105,
 106,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 503,
 508,
 509,
 511,
 601,
 602,
 603,
 604,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1201,
 1202,
 1203,
 1204,
 1205,
 1206,
 1207,
 1208,
 1209,
 1210,
 1211,
 1212,
 1213,
 1214,
 1301,
 1302,
 1401,
 1402,
 1403,
 1404,
 1501,
 1502,
 1503,
 1504,
 1505,
 1506,
 1507,
 1508,
 1509,
 1510,
 1511,
 1512,
 1513,
 1514,
 1515,
 1516,
 1517,
 1518,
 1520,
 1521,
 1522,
 1601,
 1602,
 1603,
 1604,
 1605,
 1701,
 1702,
 1703,
 1704,
 1801,
 1803,
 1804,
 1805,
 1806,
 1901,
 1902,
 1903,
 

In [102]:
# guardo la lista para levantarla desde otros notebooks
df_list = pd.DataFrame(list_bioecon_HS4dig, columns=["lista_prodbio"])
df_list.to_csv('../data/processed/list_bioecon_HS4dig.csv', index=False)

In [103]:
df_list

,lista_prodbio
0,101
1,102
2,103
3,104
4,105
...,...
575,9113
576,9401
577,9403
578,9404
